In [1]:
import xgboost as xgb
import sklearn

from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import log_loss,confusion_matrix,classification_report,roc_curve,auc,accuracy_score,roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import MultinomialNB

# Onehot encoding

from sklearn.preprocessing import LabelEncoder

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy import sparse
import gc
import os
import random

In [2]:
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    #torch.manual_seed(seed)
    #torch.cuda.manual_seed(seed)
    #torch.backends.cudnn.deterministic = True
    #torch.backends.cudnn.benchmark = False
seed_everything()

In [3]:
'''
!mkdir input/
%cd input/
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip
!unzip NewsAggregatorDataset.zip
'''

'\n!mkdir input/\n%cd input/\n!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip\n!unzip NewsAggregatorDataset.zip\n'

In [4]:
# 読込時のエラー回避のためダブルクォーテーションをシングルクォーテーションに置換
#!sed -e 's/"/'\''/g' ./input/newsCorpora.csv > ./input/newsCorpora_re.csv
#%cd ..

In [5]:


# データの読込
df = pd.read_csv('./input/newsCorpora_re.csv', header=None, sep='\t', names=['ID', 'TITLE', 'URL', 'PUBLISHER', 'CATEGORY', 'STORY', 'HOSTNAME', 'TIMESTAMP'])

# データの抽出
df = df.loc[df['PUBLISHER'].isin(['Reuters', 'Huffington Post', 'Businessweek', 'Contactmusic.com', 'Daily Mail']), ['TITLE', 'CATEGORY']]



In [9]:
df.head()

,TITLE,CATEGORY
12,Europe reaches crunch point on banking union,b
13,ECB FOCUS-Stronger euro drowns out ECB's messa...,b
19,"Euro Anxieties Wane as Bunds Top Treasuries, S...",b
20,Noyer Says Strong Euro Creates Unwarranted Eco...,b
29,REFILE-Bad loan triggers key feature in ECB ba...,b


In [10]:
# データの分割
df_train, df_valid_test = train_test_split(df, test_size=0.2, shuffle=True, random_state=123, stratify=df['CATEGORY'])
df_valid, df_test = train_test_split(df_valid_test, test_size=0.5, shuffle=True, random_state=123, stratify=df_valid_test['CATEGORY'])
df_train.reset_index(drop=True, inplace=True)
df_valid.reset_index(drop=True, inplace=True)
df_test.reset_index(drop=True, inplace=True)

print(df_train.head())

                                               TITLE CATEGORY
0  REFILE-UPDATE 1-European car sales up for sixt...        b
1  Amazon Plans to Fight FTC Over Mobile-App Purc...        t
2  Kids Still Get Codeine In Emergency Rooms Desp...        m
3  What On Earth Happened Between Solange And Jay...        e
4  NATO Missile Defense Is Flight Tested Over Hawaii        b


In [11]:
vect_word = TfidfVectorizer(max_features=20000, lowercase=True, analyzer='word',
                        stop_words= 'english',ngram_range=(1,3),dtype=np.float32)
vect_char = TfidfVectorizer(max_features=40000, lowercase=True, analyzer='char',
                        stop_words= 'english',ngram_range=(3,6),dtype=np.float32)

In [12]:
# Word ngram vector
tr_vect = vect_word.fit_transform(df_train['TITLE'])
vl_vect = vect_word.transform(df_valid['TITLE'])
ts_vect = vect_word.transform(df_test['TITLE'])

# Character n gram vector
tr_vect_char = vect_char.fit_transform(df_train['TITLE'])
vl_vect_char = vect_char.transform(df_valid['TITLE'])
ts_vect_char = vect_char.transform(df_test['TITLE'])
gc.collect()

0

In [22]:
tr_vect.shape

(10684, 20000)

In [23]:
X = sparse.hstack([tr_vect, tr_vect_char])
x_val = sparse.hstack([vl_vect, vl_vect_char])
x_test = sparse.hstack([ts_vect, ts_vect_char])

In [24]:
le = LabelEncoder()
y_tr = le.fit_transform(df_train['CATEGORY'].values)
y_vl = le.transform(df_valid['CATEGORY'].values)
y_te = le.transform(df_test['CATEGORY'].values)

In [31]:
X.shape

(10684, 60000)

In [30]:
x_val.shape

(1336, 60000)

In [26]:
y_vl.shape

(1336,)

In [27]:
y_tr.shape

(10684,)

In [32]:
model = MultinomialNB()
model.fit(X, y_tr)

MultinomialNB()

In [33]:
# 検証データを予測する
y_pred = model.predict_proba(x_val)
y_pred_max = np.argmax(y_pred, axis=1)  # 最尤と判断したクラスの値にする

accuracy = sum(y_vl == y_pred_max) / len(y_vl)
print(accuracy)

0.8727544910179641


In [34]:
print(roc_auc_score(y_vl, y_pred, multi_class='ovo'))
print(log_loss(y_vl, y_pred))

0.9488893949984073
0.44275303838453967


In [35]:
# 評価データを予測する
y_pred = model.predict_proba(x_test)
y_pred_max = np.argmax(y_pred, axis=1)  # 最尤と判断したクラスの値にする

accuracy = sum(y_te == y_pred_max) / len(y_te)
print(accuracy)

0.875748502994012


In [36]:
print(roc_auc_score(y_te, y_pred, multi_class='ovo'))
print(log_loss(y_te, y_pred))

0.9606171820491908
0.3952159299898763
